In [1]:
from typing import TypedDict, List, Annotated
class StockAnalysisState(TypedDict):
    ticker: str
    analysis_type: str
    custom_prompt: str
    fundamentals: Annotated[dict, "fundamentals"]
    graphs: Annotated[List[str], "graphs"]
    pros_cons: Annotated[str, "pros_cons"]
    news: Annotated[List[dict], "news"]

In [3]:
!pip install yfinance

     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     --- ------------------------------------ 0.3/3.0 MB ? eta -:--:--
     --- ------------------------------------ 0.3/3.0 MB ? eta -:--:--
     ---------- ----------------------------- 0.8/3.0 MB 1.4 MB/s eta 0:00:02
     ------------- -------------------------- 1.0/3.0 MB 1.3 MB/s eta 0:00:02
     ----------------- ---------------------- 1.3/3.0 MB 1.4 MB/s eta 0:00:02
     -------------------- ------------------- 1.6/3.0 MB 1.4 MB/s eta 0:00:02
     ------------------------ --------------- 1.8/3.0 MB 1.3 MB/s eta 0:00:01
     ------------------------ --------------- 1.8/3.0 MB 1.3 MB/s eta 0:00:01
     ------------------------ --------------- 1.8/3.0 MB 1.3 MB/s eta 0:00:01
     --------------------------- ------------ 2.1/3.0 MB 1.1 MB/s eta 0:00:01
     --------------------------- ------------ 2.1/3.0 MB 1.1 MB/s eta 0:00:01
     --------------------------- ------------ 2.1/3.0 MB 1.1 MB/s eta 0:00:01
     --

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyppeteer 2.0.0 requires websockets<11.0,>=10.0, but you have websockets 15.0.1 which is incompatible.


In [4]:
import yfinance as yf
def fetch_stock_data(state: StockAnalysisState):
    stock = yf.Ticker(state["ticker"])
    return {
        "fundamentals": {
            "Market Cap": stock.info.get("marketCap"),
            "P/E Ratio": stock.info.get("trailingPE"),
            "Revenue": stock.info.get("totalRevenue"),
            "EPS": stock.info.get("trailingEps"),
            "Debt-to-Equity": stock.info.get("debtToEquity"),
        }
    }

In [5]:
import matplotlib.pyplot as plt
def generate_stock_graphs(state: StockAnalysisState):
    stock = yf.Ticker(state["ticker"])
    hist = stock.history(period="1y")
    plt.figure(figsize=(10, 5))
    plt.plot(hist.index, hist["Close"], label="Closing Price", color="blue")
    plt.legend()
    plt.title(f"{state['ticker']} Price Trend")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.grid()
    chart_filename = f"{state['ticker']}_chart.png"
    plt.savefig(chart_filename)
    plt.close()
    return {"graphs": [chart_filename]}
def generate_stock_graphs(state: StockAnalysisState):
    stock = yf.Ticker(state["ticker"])
    hist = stock.history(period="1y")
    plt.figure(figsize=(10, 5))
    plt.plot(hist.index, hist["Close"], label="Closing Price", color="blue")
    plt.legend()
    plt.title(f"{state['ticker']} Price Trend")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.grid()
    chart_filename = f"{state['ticker']}_chart.png"
    plt.savefig(chart_filename)
    plt.close()
    return {"graphs": [chart_filename]}

In [6]:
import ollama
import json
def analyze_pros_cons(state: StockAnalysisState):
    prompt = (
        f"Analyze this stock as a {state['analysis_type']} investment:\n"
        f"{json.dumps(state['fundamentals'], indent=2)}\n\n"
        f"Custom Input: {state['custom_prompt']}"
    )
    response = ollama.chat(model="llama3.2", messages=[{"role": "user", "content": prompt}])
    return {"pros_cons": response["message"]["content"]}

In [7]:
import requests
def fetch_news(state: StockAnalysisState):
    api_key = "1c0e4baf1526438fad8660a8a8893582"
    url = f"https://newsapi.org/v2/everything?q={state['ticker']}&apiKey={api_key}"
    response = requests.get(url).json()
    news_articles = [{"title": article["title"], "url": article["url"]} 
                     for article in response.get("articles", [])[:5]]
    return {"news": news_articles}

In [8]:
from langgraph.graph import StateGraph
graph = StateGraph(StockAnalysisState)
graph.add_node("fetch_stock_data", fetch_stock_data)
graph.add_node("generate_stock_graphs", generate_stock_graphs)
graph.add_node("analyze_pros_cons", analyze_pros_cons)
graph.add_node("fetch_news", fetch_news)
graph.set_entry_point("fetch_stock_data")
graph.add_edge("fetch_stock_data", "generate_stock_graphs")
graph.add_edge("generate_stock_graphs", "analyze_pros_cons")
graph.add_edge("analyze_pros_cons", "fetch_news")
app = graph.compile()

In [9]:
def run_analysis(ticker, analysis_type="general", custom_prompt=""):
    initial_state = StockAnalysisState(
        ticker=ticker,
        analysis_type=analysis_type,
        custom_prompt=custom_prompt,
        fundamentals={},
        graphs=[],
        pros_cons="",
        news=[]
    )
    return app.invoke(initial_state)

In [10]:
if __name__ == "__main__":
    result = run_analysis("AAPL", "growth", "Focus on long-term investment potential.")
    print(json.dumps(result, indent=2))

{
  "ticker": "AAPL",
  "analysis_type": "growth",
  "custom_prompt": "Focus on long-term investment potential.",
  "fundamentals": {
    "Market Cap": 3002095632384,
    "P/E Ratio": 31.357256,
    "Revenue": 400366010368,
    "EPS": 6.41,
    "Debt-to-Equity": 146.994
  },
  "graphs": [
    "AAPL_chart.png"
  ],
  "pros_cons": "Based on the provided stock analysis, here's a comprehensive evaluation of its long-term growth potential:\n\n**Key Indicators:**\n\n1. **Market Cap:** $2.3 Trillion - A large market capitalization indicates significant scale and resources for the company.\n2. **P/E Ratio:** 31.4x - The price-to-earnings ratio is higher than the industry average, suggesting that investors are expecting high future growth from the company.\n3. **Revenue:** $400 billion - A substantial revenue stream indicates a strong business model with significant market share.\n4. **EPS (Earnings Per Share):** $6.41 - High EPS suggests profitability and potentially attractive dividend paymen